Compute the Annual Search Fraction similarity as described by Lee et al. (2015)

    Lee, C.M.C., et al., Search-based peer firms: Aggregating investor perceptions through internet co-searches. Journal of Financial Economics (2015).
    
Because the calculation can be broken down to the daily level, for the purpose of memory efficiency, we will count unique combinations of `(ip, cik, next cik)` for each day, and then sum the daily counts to the annual level. 

In [153]:
import time
import os
import dask
import pickle

import pandas as pd
import numpy as np

from dask.diagnostics import ProgressBar
from calendar import monthrange
from datetime import datetime
from zipfile import ZipFile
from io import BytesIO
from tqdm import tqdm

In [3]:
# store large data files in separated dir
from Utility.dir_LF import large_file_dir

# keep form 
from Utility.Form_keep import keep

# Prepare Data

## Masterindex

In [76]:
def append_mi_pd(year):
    pd_lst = []
    for y in range(1995,year): 
        print('appending',y)
        for quarter in range(1,5):
            
            mi_fp = f"{large_file_dir}/masterindex/mi_{y}_{quarter}.csv"
            mi_pd = pd.read_csv(mi_fp)
            mi_pd['accession']= mi_pd.apply(lambda row: row['Filename'].split('.txt')[0].split('/')[3], axis = 1) # create new column with accession
            pd_lst.append(mi_pd)
            
    mi_pd = pd.concat(pd_lst, axis=0, ignore_index=True)
    mi_pd = mi_pd[['accession', 'Form Type']].drop_duplicates()
    mi_pd.to_csv(f"data/mis{year}.csv")
    
    return

In [77]:
append_mi_pd(2004)

appending 1995
appending 1996
appending 1997
appending 1998
appending 1999
appending 2000
appending 2001
appending 2002
appending 2003


## SP1500 index

In [25]:
def getSPindex(day):
    '''
    get the S&P index as of `day`
    
    day: a python datetime instance
    
    return two dictionaries
     sp_d = {
       500: set(int,...), # set of cik firms that are in the S&P 500 universe
       1500: set(int,...) # set of cik firms that are in the S&P 1500 universe
     }
     cik2gvkey_d = {
       500: a pandas series with cik as index and gvkey as values, 
            the map from cik to gvkey in S&P500
       1500: a pandas series with cik as index and gvkey as values, 
            the map from cik to gvkey in S&P1500
     }
    '''
    # see `./data/index_constituents-compustat_north_america.pdf` for how
    # 'SPIndex.csv' is generated and should be used
    sp_index_pd = pd.read_csv('./data/SPIndex.csv', parse_dates=['from', 'thru'])
    sp_500_pd = sp_index_pd.loc[sp_index_pd.tic=='I0003',:]
    sp_1500_pd = sp_index_pd.loc[sp_index_pd.tic=='I0020',:]
    sp_d = {}
    cik2gvkey_d = {}
    for universe_size, tmp in zip([500, 1500], [sp_500_pd, sp_1500_pd]):
        tmp = tmp[(tmp['from'] <= day) & ((tmp.thru >= day)|tmp.thru.isna())]
        # S&P 1500 at a given timepoint might not consist of exactly 1500 unique cik firms
        # but should roughly be 1500, the same for S&P 500
        # assert tmp.co_cik.nunique() == universe_size
        tmp2 = tmp.co_cik.unique().tolist() # only get cik
        sp_co_cik_l = [int(x) for x in tmp2 if str(x) != 'nan']
        sp_d[universe_size] = sp_co_cik_l
        tmp3 = tmp[['gvkey', 'co_cik']].drop_duplicates()
        tmp3 = tmp3.set_index('co_cik')
        cik2gvkey_d[universe_size] = tmp3
    return sp_d, cik2gvkey_d

In [27]:
sp_d, cik2gvkey_d = getSPindex(datetime(2008,1,1))

# Data Filtering

In [53]:
def get_daily_log_pd(day):
    '''
    day: python datetime instance
    '''
    save_fp = f"{large_file_dir}/log/{day.year}/log{day.strftime('%Y%m%d')}.zip"
    with ZipFile(save_fp) as f:
        which = f.namelist()[0]
        df = pd.read_csv(BytesIO(f.read(which)), encoding='utf-8')
        
    df['date'] = df['date'].astype('datetime64[ns]')
    df['cik'] = df['cik'].astype(int)
    df.drop(['zone','extention','code','size','idx','norefer','noagent','find','browser'], axis=1, inplace=True)
    return df

In [79]:
mis = pd.read_csv('data/mis2004.csv')

In [ ]:
# have to prepare global vars: sp_d, keep


@dask.delayed
def get_pairs_day(day):
    
    '''
    day: a datetime instance
    e.g. : datetime(2008, 1, 1)
    '''
    
    df = get_daily_log_pd(day)
    if df.shape[0] != 0:
    #print('processing'+str())
        
    
        df = df.merge(mis, on='accession')  # get Form Type from master index(1995-2011)
# Step2 : SP1500
        df = df[df.cik.isin(sp_d[1500])]

# Step3&4 : unique cik >1, cik<50
        df_freq = df.groupby(by='ip', as_index=False).agg({'cik': pd.Series.nunique})
        df_freq.columns = ['ip','uniq_search']  # unique cik search for each ip
        df = df.merge(df_freq, on='ip')
        del df_freq
        df = df.drop(df[(df['uniq_search'] > 50) | (df['uniq_search'] <= 1)].index)

# Step5: keep certain forms searches
    
        df = df[df['Form Type'].isin(keep)]

# Step5a: filter again those only 1 obs 
        #* this was not mentioned in Table 1, but illustrated in Table A1 (reason for dropping observation 3)

        df_freq2 = df.groupby(by='ip', as_index=False).agg({'cik': pd.Series.nunique})
        df_freq2.columns = ['ip','uniq_search2']  # unique cik search for each ip
        df = df.merge(df_freq2, on='ip')
        del df_freq2
        df = df.drop(df[(df['uniq_search2'] <= 1)].index)
        df.drop(['uniq_search','crawler'],axis= 1, inplace = True)  # drop some unused vars

# Step 6a, Removing consecutive searches
        df = df[['ip','date','time','cik']]  # naturally ordered by ip/date/time
        df = df.groupby('ip').apply(
            lambda x: x.loc[
                x.cik.shift(-1) != x.cik
                ]).reset_index(drop = True)

#  step 6b, removing double-counted

        ip_list = df['ip'].unique().tolist()

        a_day_cik_link = []
        for ip in ip_list:

            df_temp = df[df['ip'] == ip]

            temp_link = [] # a _user_cik_link
            for i in range(len(df_temp['cik'].tolist())-1):
                a_link = (df_temp['cik'].tolist()[i],df_temp['cik'].tolist()[i+1]) # tuple ,hashable
                temp_link.append(a_link)
            temp_link = list(set(temp_link)) # removing double-counted

            a_day_cik_link = a_day_cik_link + temp_link
        return a_day_cik_link
    #cik_list_day = df['cik'].unique().tolist()
    #print('sample pairs', a_day_cik_link[0:10])

    # output is a day cik link

In [ ]:
def get_pairs_year(year): # Always Using SP1500 index of 2008 Jan 1

    ayear_pairs = []

    for month in range(1,13):
        last_day = monthrange(year, month)[1]
        for day in range(last_day):
            try:
                aday= get_pairs_day(datetime(year, month, day+1))
                ayear_pairs = ayear_pairs + aday
            except:
                continue
    with ProgressBar():   
        ayear_pairs = dask.compute(ayear_pairs)[0]
    
    return ayear_pairs

In [ ]:
pairs2003 = get_pairs_year(2003)

df is empty in 20030101
df is empty in 20030104
df is empty in 20030105
df is empty in 20030111
df is empty in 20030112
df is empty in 20030117
df is empty in 20030118
df is empty in 20030119
df is empty in 20030120
df is empty in 20030121
df is empty in 20030122
df is empty in 20030131
df is empty in 20030201
df is empty in 20030202
df is empty in 20030203
df is empty in 20030204
df is empty in 20030205
df is empty in 20030206
df is empty in 20030207
df is empty in 20030208
df is empty in 20030209
df is empty in 20030210
df is empty in 20030211
df is empty in 20030218


In [ ]:
def get_asf(pairs, year):
    df = pd.DataFrame(pairs, columns=['a', 'b'])
    #df2008.info()
    #  freq_a
    df2 = df.groupby('a').size().reset_index(name="freq_a")
    #df2.info()
# add freq_a
    df3 = df2008.merge(df2, on="a")
    #df3.info()
    # freq_ab
    df4 = df3.groupby(["a", "b"]).size().reset_index(name="freq_ab")
    #df4.info()

# add freq_ab
    df5 = df3.merge(df4, on =['a','b'])
    df5['asf']=df5['freq_ab']/df5['freq_a']
    df5.drop(['freq_ab','freq_a'], axis= 1, inplace = True) 
    
    df5.to_csv(f'asf{year}.csv',, sep='\t', encoding='utf-8')
    return df5

In [ ]:
df = get_asf(pairs2003, 2003)
df[0:10]